# Importations et version de TensorFlow

In [61]:
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

# Afficher la version de TensorFlow
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.17.1


Fonction pour charger le fichier et afficher les premières lignes

In [62]:
def load_doc(url):
    df = pd.read_csv(url, delimiter='\t', header=None)
    return df

doc = load_doc('https://go.aws/38ECHUB')
print(doc.head())

      0           1
0   Go.        Va !
1   Hi.     Salut !
2  Run!     Cours !
3  Run!    Courez !
4  Wow!  Ça alors !


Limitation du dataset pour éviter des problèmes de mémoire

In [63]:
# Réduction de la taille du dataset
doc = doc.iloc[:5000, :]
print("Nombre de lignes après réduction:", len(doc))

Nombre de lignes après réduction: 5000


Ajout des tokens <start> et <end>

In [64]:
def begin_end_sentence(sentence):
    return '<start> ' + sentence + ' <end>'

doc.iloc[:, 0] = doc.iloc[:, 0].apply(begin_end_sentence)
doc.iloc[:, 1] = doc.iloc[:, 1].apply(begin_end_sentence)

Tokenisation des phrases

In [65]:
# Initialisation des tokenizers
tokenizer_fr = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tokenizer_en = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')

# Apprentissage des tokenizers
tokenizer_en.fit_on_texts(doc.iloc[:, 0])
tokenizer_fr.fit_on_texts(doc.iloc[:, 1])

Conversion en indices et padding

In [66]:
# Conversion des textes en indices
doc['fr_indices'] = tokenizer_fr.texts_to_sequences(doc.iloc[:, 1])
doc['en_indices'] = tokenizer_en.texts_to_sequences(doc.iloc[:, 0])

# Padding des séquences
padded_fr_indices = tf.keras.preprocessing.sequence.pad_sequences(doc['fr_indices'], padding='post')
padded_en_indices = tf.keras.preprocessing.sequence.pad_sequences(doc['en_indices'], padding='post')

Création des datasets TensorFlow

In [67]:
fr_ds = tf.data.Dataset.from_tensor_slices(padded_fr_indices)
en_ds = tf.data.Dataset.from_tensor_slices(padded_en_indices)

tf_ds = tf.data.Dataset.zip((fr_ds, en_ds))

# Paramètres globaux
BATCH_SIZE = 50
TAKE_SIZE = int(0.7 * len(doc))
steps_per_epoch = TAKE_SIZE
BUFFER_SIZE = TAKE_SIZE

train_data = tf_ds.take(TAKE_SIZE).shuffle(TAKE_SIZE).batch(BATCH_SIZE)
test_data = tf_ds.skip(TAKE_SIZE).batch(BATCH_SIZE)

Initialisation des dimensions

In [68]:
# Dimensions pour les modèles
embedding_dim = 256
units = 1024
vocab_inp_size = len(tokenizer_fr.word_index) + 1
vocab_tar_size = len(tokenizer_en.word_index) + 1

Définition de l’encodeur

In [93]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            self.enc_units,
            return_sequences=True,
            return_state=True,
            recurrent_initializer='glorot_uniform'
        )

    def call(self, x, hidden):
        x = self.embedding(x)
        outputs = self.gru(x, initial_state=hidden)  # GRU retourne plusieurs valeurs
        print(f"GRU outputs: {outputs}")  # Affichez toutes les sorties pour identifier ce qui est retourné

        return outputs

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

# Initialisation de l'encodeur
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

In [94]:
sample_input = tf.random.uniform((BATCH_SIZE, 12), maxval=vocab_inp_size, dtype=tf.int32)
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_state = encoder(sample_input, sample_hidden)

print("Sample Output Shape:", sample_output.shape)  # (batch_size, sequence_length, units)
print("Sample Hidden State Shape:", sample_state.shape)  # (batch_size, units)

GRU outputs: (<tf.Tensor: shape=(50, 12, 1024), dtype=float32, numpy=
array([[[ 2.9441467e-03,  2.7527760e-03,  1.0442005e-03, ...,
         -4.3323333e-03,  1.6991511e-03,  5.3308811e-03],
        [ 5.4919859e-03,  7.6247845e-03, -2.1558024e-03, ...,
         -7.0944251e-03,  1.3625396e-03, -8.8715487e-06],
        [ 5.4178829e-04,  8.1924219e-03, -8.5329805e-03, ...,
         -2.9688666e-03, -6.1635356e-03,  2.4896732e-03],
        ...,
        [ 3.5478466e-03,  1.1088207e-02, -6.3877204e-03, ...,
          1.0711063e-02, -1.2087104e-03,  1.9639463e-03],
        [ 1.0691530e-02,  3.9292807e-03,  6.7420624e-04, ...,
         -3.8237782e-04,  3.6534094e-03, -1.2789568e-03],
        [ 2.9791386e-03, -1.2027364e-03, -9.4449334e-03, ...,
         -1.9052182e-03, -4.4729183e-03, -3.8419939e-03]],

       [[-3.8755266e-03, -5.2534374e-03, -3.5277419e-03, ...,
         -7.6274909e-03,  4.3038046e-03,  9.1923593e-04],
        [ 9.2692925e-03, -1.2041737e-04, -4.4423244e-03, ...,
         -2.8

ValueError: too many values to unpack (expected 2)

Ajout de l’attention

In [70]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # Étendre la dimension de query pour correspondre à values
        hidden_with_time_axis = tf.expand_dims(query, 1)  # (batch_size, 1, units)

        # Calculer le score d'attention
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # Calculer les poids d'attention
        attention_weights = tf.nn.softmax(score, axis=1)

        # Calculer le vecteur de contexte
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)  # (batch_size, units)

        return context_vector, attention_weights

Définition du décodeur

In [71]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        x = self.fc(output)
        return x, state, attention_weights

# Initialisation du décodeur
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

Fonction de coût et optimisation

In [72]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

Entraînement

In [73]:
optimizer = tf.keras.optimizers.Adam()

#@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims(targ[:, 0], 1)

        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = loss / int(targ.shape[1])
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

Boucle d’entraînement

In [77]:
import time

EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(train_data):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
            print(f'Epoch {epoch + 1} Batch {batch} Loss {batch_loss.numpy():.4f}')

    print(f'Epoch {epoch + 1} Loss {total_loss / steps_per_epoch:.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec\n')

ValueError: Exception encountered when calling Encoder.call().

[1mtoo many values to unpack (expected 2)[0m

Arguments received by Encoder.call():
  • x=tf.Tensor(shape=(50, 12), dtype=int32)
  • hidden=tf.Tensor(shape=(50, 1024), dtype=float32)

In [81]:
sample_input = tf.random.uniform((BATCH_SIZE, 12), maxval=vocab_inp_size, dtype=tf.int32)
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_state = encoder(sample_input, sample_hidden)

print("Sample Output Shape:", sample_output.shape)  # (batch_size, sequence_length, units)
print("Sample Hidden State Shape:", sample_state.shape)  # (batch_size, units)

ValueError: Exception encountered when calling Encoder.call().

[1mtoo many values to unpack (expected 2)[0m

Arguments received by Encoder.call():
  • x=tf.Tensor(shape=(50, 12), dtype=int32)
  • hidden=tf.Tensor(shape=(50, 1024), dtype=float32)